#### 데이터 로드하기

In [11]:
import pandas as pd
import numpy as np
import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('./data/customer.csv')
  hotel_tb = pd.read_csv('./data/hotel.csv')
  reserve_tb = pd.read_csv('./data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('./data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('./data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('./data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('./awesomebook-master/data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = pd.read_csv('./data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('./data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [4]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

### 3. 여러 테이블 합치기

#### 1) 레코드 테이블과 마스터 테이블의 결합

In [ ]:
pd.merge(reserve_tb.query('people_num == 1'),   # 숙박 인원수가 1명인 예약 추출
         hotel_tb.query('is_business'),         # 영업 중인 호텔 찾기
         on='hotel_id', how='inner')            # 이너 조인으로 인원 수 1명인 호텔 목록 찾기

## 쿼리를 통해 각 테이블에서 조건 필터링한 후, 호텔 아이디(hotel_id)를 바탕으로 합침

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,True
1,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.595248,130.633567,True
2,r13,h_223,c_2,2017-10-19 03:03:30,2017-10-21,09:30:00,2017-10-23,1,137000,68500,C,C-2,38.329097,140.698165,True
3,r18,h_132,c_3,2016-10-22 02:18:48,2016-11-12,12:00:00,2016-11-13,1,20400,20400,C,C-1,38.231842,140.797268,True
4,r25,h_277,c_4,2016-03-28 07:17:34,2016-04-07,10:30:00,2016-04-10,1,39300,13100,C,C-1,38.233985,140.795603,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,r3999,h_72,c_993,2016-06-09 04:35:48,2016-07-04,12:00:00,2016-07-07,1,29100,9700,B,B-2,35.545712,139.596157,True
442,r4001,h_253,c_993,2017-01-20 18:50:35,2017-02-10,11:00:00,2017-02-12,1,10400,5200,C,C-3,38.430220,140.696571,True
443,r4005,h_150,c_994,2016-03-25 11:51:26,2016-04-02,09:00:00,2016-04-03,1,25200,25200,B,B-2,35.546229,139.693687,True
444,r4011,h_137,c_995,2017-08-13 19:23:48,2017-09-07,09:30:00,2017-09-09,1,12400,6200,F,F-1,34.538252,132.469507,True


#### 2) 마스터 테이블을 조건에 따라 변경하기

목표    
호텔 추천 시스템을 위해 **지역별 호텔 수에 따라 결합 키를 동적으로 생성**하는 것입니다.

핵심 로직
- `small_area_name` 기준 호텔 수(`hotel_cnt`)를 계산
- **20개 이상**: `small_area_name`을 `join_area_id`로 사용 (세분화된 지역)
- **20개 미만**: `big_area_name`을 `join_area_id`로 사용 (넓은 지역)

작업 흐름
```
hotel_tb → small_area_name별 호텔 수 계산 → 조건부 join_area_id 생성
    ↓
base_hotel_mst (hotel_id, join_area_id)
    ↓
recommend_hotel_mst 생성 → join_area_id로 결합 → 같은 지역 호텔 추천

예시
- 강남구(50개) → 강남구 내 다른 호텔 추천
- 시골 마을(3개) → 경기도 전체에서 추천

In [6]:
import gc

In [12]:
# 1. small_area_name 기준 호텔 수 계산
small_area_mst = hotel_tb \
    .groupby(['big_area_name', 'small_area_name'], as_index=False) \
    .size().reset_index()
small_area_mst.head()


,index,big_area_name,small_area_name,size
0,0,A,A-1,35
1,1,A,A-3,30
2,2,B,B-1,15
3,3,B,B-2,18
4,4,B,B-3,19


In [13]:
# 호텔 영역별 호텔 개수(hotel_cnt) 생성
small_area_mst.columns = ['index', 'big_area_name',	'small_area_name', 'hotel_cnt']
small_area_mst.head()


,index,big_area_name,small_area_name,hotel_cnt
0,0,A,A-1,35
1,1,A,A-3,30
2,2,B,B-1,15
3,3,B,B-2,18
4,4,B,B-3,19


In [15]:
# 2. 조건부 join_area_id 생성 (20개 기준)
## 호텔 20개 이상은 작은 영역 이름을 쓰고, 20개 이하는 큰 영역 이름을 쓴다
small_area_mst['join_area_id'] = np.where(small_area_mst['hotel_cnt'] - 1 >= 20, 
                                          small_area_mst['small_area_name'],
                                          small_area_mst['big_area_name'])
small_area_mst.head()

,index,big_area_name,small_area_name,hotel_cnt,join_area_id
0,0,A,A-1,35,A-1
1,1,A,A-3,30,A-3
2,2,B,B-1,15,B
3,3,B,B-2,18,B
4,4,B,B-3,19,B


In [ ]:
# 조인한 id를 최종 사용할 것이므로 hotel_cnt, big_area_name은 삭제. (small_area_name은 이따 사용할 것)
small_area_mst.drop(['hotel_cnt', 'big_area_name'], axis=1, inplace=True)
small_area_mst.head()

,index,small_area_name,join_area_id
0,0,A-1,A-1
1,1,A-3,A-3
2,2,B-1,B
3,3,B-2,B
4,4,B-3,B


In [17]:
# 3. 호텔-지역 매핑 테이블 생성
base_hotel_mst = pd.merge( hotel_tb, small_area_mst, #호텔 데이터베이스와 호텔 개수, 영역 계산한 small 테이블 결합
                           on='small_area_name'      #small area name을 기준으로 병합
                          ).loc[:, ['hotel_id', 'join_area_id']]    #병합한 테이블 중 호텔 아이디, 호텔이 속한 지역 아이디 열만 추출
base_hotel_mst.head()

,hotel_id,join_area_id
0,h_1,D
1,h_2,A-1
2,h_3,E
3,h_4,C-3
4,h_5,G


In [18]:
# 병합했으니 불필요한 변수 제거
del small_area_mst
gc.collect()

0

In [ ]:
# 4. 호텔 추천 후보 마스터 테이블 제작
recommend_hotel_mst = pd.concat([
     hotel_tb[['small_area_name', 'hotel_id']].rename(columns={'small_area_name' : 'join_area_id'}, inplace=False),
     hotel_tb[['big_area_name', 'hotel_id']].rename(columns={'big_area_name' : 'join_area_id'}, inplace=False)
     ])
recommend_hotel_mst.head(2)

,join_area_id,hotel_id
0,D-2,h_1
1,A-1,h_2


In [20]:
recommend_hotel_mst.rename(columns={'hotel_id' : 'rec_hotel_id'}, inplace=True)
recommend_hotel_mst.head(1)

,join_area_id,rec_hotel_id
0,D-2,h_1


In [ ]:
# 5. 호텔에 따른 추천 호텔 테이블 만들기 
pd.merge( base_hotel_mst, recommend_hotel_mst, 
          on='join_area_id') \
          .loc[ : , ['hotel_id','rec_hotel_id']] \
          .query('hotel_id != rec_hotel_id')

,hotel_id,rec_hotel_id
1,h_1,h_14
2,h_1,h_22
3,h_1,h_27
4,h_1,h_40
5,h_1,h_45
...,...,...
9658,h_300,h_247
9659,h_300,h_256
9660,h_300,h_257
9661,h_300,h_277


#### 3) 과거 데이터를 사용하기 위한 준비

In [ ]:
# 1. 고객 별로 시간 순대로 결제 데이터 정렬하기
# 1. .shift() 사용하기
# 고객 id 별로 그룹화, 예약 시간 순서대로 정렬,
result = reserve_tb.groupby('customer_id').apply(lambda group:
                                                 group.sort_values(by='reserve_datetime', axis=0, inplace=False)) # 행 기준 정렬
result.head()

/var/folders/z1/lkyzpxrd2zl49qz1984s_0nr0000gn/T/ipykernel_2623/1207767368.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = reserve_tb.groupby('customer_id').apply(lambda group:


reserve_id hotel_id customer_id     reserve_datetime  \
customer_id                                                          
c_1         0         r1     h_75         c_1  2016-03-06 13:09:42   
            1         r2    h_219         c_1  2016-07-16 23:39:55   
            2         r3    h_179         c_1  2016-09-24 10:03:17   
            3         r4    h_214         c_1  2017-03-08 03:20:10   
            4         r5     h_16         c_1  2017-09-05 19:50:37   

              checkin_date checkin_time checkout_date  people_num  total_price  
customer_id                                                                     
c_1         0   2016-03-26     10:00:00    2016-03-29           4        97200  
            1   2016-07-20     11:30:00    2016-07-21           2        20600  
            2   2016-10-19     09:00:00    2016-10-22           2        33600  
            3   2017-03-29     11:00:00    2017-03-30           4       194400  
            4   2017-09-22     10:30:00    2017-09-23           3        68100

In [27]:
# 열을 특정 위치로 이동
result['before_price'] = pd.Series(result['total_price'].shift(periods = 2))
result.head(2)

reserve_id hotel_id customer_id     reserve_datetime  \
customer_id                                                          
c_1         0         r1     h_75         c_1  2016-03-06 13:09:42   
            1         r2    h_219         c_1  2016-07-16 23:39:55   

              checkin_date checkin_time checkout_date  people_num  \
customer_id                                                         
c_1         0   2016-03-26     10:00:00    2016-03-29           4   
            1   2016-07-20     11:30:00    2016-07-21           2   

               total_price  before_price  
customer_id                               
c_1         0        97200           NaN  
            1        20600           NaN

#### 4) 과거 n일 동안의 예약 가격 합계 구하기

In [29]:
# 2. 고객 별로 해당 시간 이전까지 결제했던 금액의 합계 구하기 -> before_price 열에 넣기
# 2. .rolling(center= , window = , min_periods = ) 사용하기

# 일단 예약 시간 기준으로 정렬하기 (아까 반복)
result = reserve_tb.groupby('customer_id') \
                   .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True)) \
                   .reset_index(drop=True)

# 합계 구해서 넣기
result['price_sum'] = pd.Series(
    result.loc[ : , ['customer_id', 'total_price']]
    .groupby('customer_id')
    .rolling( center = False, window = 3, min_periods = 3).sum()\
    .reset_index(drop=True)\
    .loc[ : , 'total_price']
)
result.head(4)

/var/folders/z1/lkyzpxrd2zl49qz1984s_0nr0000gn/T/ipykernel_2623/1301439837.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = reserve_tb.groupby('customer_id') \


,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,NaN
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,151400.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,248600.0


In [45]:
# 3. 과거 n건의 평균값 구하기
result = reserve_tb.groupby('customer_id') \
         .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True))\
         .reset_index(drop=True)

## 평균값 구해서 새로운 칼럼에 넣기
result['price_avg'] = pd.Series(
    result.groupby('customer_id')['total_price'].rolling(center=False, window=3, min_periods=1).mean()
    .reset_index(drop=True)
)

result['price_avg'] = result.groupby('customer_id')['price_avg'].shift(periods=1)

result.head(1)

/var/folders/z1/lkyzpxrd2zl49qz1984s_0nr0000gn/T/ipykernel_2623/2647106153.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = reserve_tb.groupby('customer_id') \


,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_avg
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN


In [ ]:
# 4. 자신은 포함하지 않으면서 같은 고객의 지난 90일간의 합계 예약 정보
# 과거 90일 간의 total_price 합계 계산 후, reserve_id 키를 기준으로 결합

# 관련 패키지 임포트하기
import pandas.tseries.offsets as offsets
import operator

# 일시형으로 변환
# 현재 데이터타입 확인 결과 object 타입 -> datatime 타입으로 변경
# reserve_tb.dtypes
reserve_tb['reserve_datetime'] = pd.to_datetime(reserve_tb['reserve_datetime'], format = '%Y-%m-%d %H:%M:%S')
reserve_tb.dtypes


reserve_id                  object
hotel_id                    object
customer_id                 object
reserve_datetime    datetime64[ns]
checkin_date                object
checkin_time                object
checkout_date               object
people_num                   int64
total_price                  int64
dtype: object

In [ ]:
# 필요한 컬럼만 가지고 sum_table 테이블 만들기 
sum_table = pd.merge(
    reserve_tb[['reserve_id', 'customer_id', 'reserve_datetime']],
    reserve_tb[['customer_id', 'reserve_datetime', 'total_price']].rename(columns={'reserve_datetime' : 'reserve_datetime_before'}),
    on='customer_id'
)
sum_table.head(1)

,reserve_id,customer_id,reserve_datetime,reserve_datetime_before,total_price
0,r1,c_1,2016-03-06 13:09:42,2016-03-06 13:09:42,97200


In [46]:
# 데이터 필터링 후 이전 90일 간 합계 산출해서 붙이기
sum_table = sum_table[ operator.and_(
    sum_table['reserve_datetime'] > sum_table['reserve_datetime_before'],
    sum_table['reserve_datetime'] + offsets.Day(-90) <= sum_table['reserve_datetime_before']
)].groupby('reserve_id')['total_price'].sum().reset_index()

sum_table.columns = ['reserve_id', 'total_price_sum']
sum_table.head(2)

,reserve_id,total_price_sum
0,r1000,44700
1,r1004,333600


In [48]:
pd.merge(reserve_tb, sum_table, on='reserve_id', how='left').fillna(0)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,0.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,20600.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,0.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,0.0
...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,0.0
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,0.0
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,0.0
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,0.0


#### 5) 고객별 과거 n일 간 예약 레코드 생성 및 병합하기

크로스 조인 사용    
* 양쪽 테이블 모두 조합
* 집계, 학습 데이터 만들기 위한 전처리에 사용 : 데이터 부족한 거보다 만은게 나으므로

In [50]:
import datetime
from dateutil.relativedelta import relativedelta

month_mst = pd.DataFrame({
    'year_month': 
        [(datetime.date(2017, 1, 1) + relativedelta(months=x)).strftime("%Y%m")
         for x in range(0, 3)]
})

# 조인 시 없는 값을 0으로 설정 -> 모든 조합을 만들되, null값은 없도록 
customer_tb['join_key'] = 0
month_mst['join_key'] = 0

customer_mst = pd.merge(
    customer_tb[['customer_id', 'join_key']], month_mst, on='join_key'
)
customer_mst.head(10)

,customer_id,join_key,year_month
0,c_1,0,201701
1,c_1,0,201702
2,c_1,0,201703
3,c_2,0,201701
4,c_2,0,201702
5,c_2,0,201703
6,c_3,0,201701
7,c_3,0,201702
8,c_3,0,201703
9,c_4,0,201701
